In [1]:
# Import data processing libraries
import pandas as pd
import numpy as np

#Import text processing libraries
import re
import urllib

# Import NLP libraries
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import contractions

# Import functions from local scripts
import sys
sys.path.insert(1, './scripts/development')
import scripts.development.preprocessing as pre

C:\Python\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
# Determine file encoding of dataset
# Only need to be run once

#get_file_encoding("../data/twitter16m.csv")

# Detector gave me utf-8 encoding, which gives an error, so I will use the value from the tutorial

In [ ]:
# Read the dataset from the csv file and check it
# Using names= to give meaningful names to columns
twitter_df = pd.read_csv("../data/twitter16m.csv", encoding="latin1", header=None, names=["sentiment", "id", "date", "query_type", "author", "message"])

In [ ]:
twitter_df.head()

In [ ]:
# Keep only the data we are interested in
twitter_df = twitter_df[["sentiment", "message"]]
twitter_df.head()

In [ ]:
# Check what kind of sentiments we have
twitter_df['sentiment'].value_counts()

### Word counts

In [ ]:
# Get the word count for every message
twitter_df.loc[:, "word_count"] = twitter_df["message"].apply(lambda x: len(x.split()))

In [ ]:
twitter_df.head()

### Character count

In [ ]:
# Get the character count for every message
twitter_df.loc[:, "character_count"] = twitter_df["message"].apply(lambda x: len(x))

In [ ]:
twitter_df.head()

### Average word length

In [ ]:
# Get the average word length for every message
twitter_df.loc[:, "avg_word_len"] = twitter_df["message"].apply(lambda x: pre.get_avg_word_len(x))

In [ ]:
twitter_df.head()

### Stop words count

In [ ]:
# Get the number of stop words in each message
twitter_df.loc[:, "stop_words_count"] = twitter_df["message"].apply(lambda x: len([word for word in x.lower().split() if word in STOP_WORDS]))

In [ ]:
twitter_df.head()

### #HashTags count

In [ ]:
# Get the number of hashtags in each message
twitter_df.loc[:, "hashtags_count"] = twitter_df["message"].apply(lambda x: pre.get_hashtags_count(x))

In [ ]:
twitter_df.head()

### Numeric counts

In [ ]:
# Get the number of numerical text in each message
twitter_df.loc[:, "numeric_count"] = twitter_df["message"].apply(
    lambda x: len([word for word in x.split() if word.isnumeric()]))

In [ ]:
twitter_df.head()

### UPPER case words count

In [ ]:
# Get the number of upper case words in each message
twitter_df.loc[:, "upper_case_count"] = twitter_df["message"].apply(lambda x: pre.get_upper_case_count(x))

In [ ]:
twitter_df.head()

### Randoms check to see if preprocessing is ok so far

In [ ]:
print(twitter_df.loc[632]["message"], "\n\n", twitter_df.loc[632])

## Data cleaning and preprocessing

### Lowercase conversion

In [ ]:
# Convert the whole message to lowercase
twitter_df.loc[:, "message"] = twitter_df["message"].apply(lambda x: x.lower())

In [ ]:
twitter_df.head()

### Contraction to expansion

In [ ]:
%%time
twitter_df.loc[:, "message"] = twitter_df["message"].apply(lambda x: pre.expand_contractions(x))

In [ ]:
twitter_df.head()

### Count and remove emails

In [ ]:
# Get the number of emails for each message
twitter_df.loc[:, "email_count"] = twitter_df["message"].apply(lambda x: pre.get_email_count(x))

In [ ]:
# Remove the emails from the message
twitter_df.loc[:, "message"] = twitter_df["message"].apply(lambda x: pre.remove_emails(x))

In [ ]:
twitter_df.head()

### Count and remove URLs

In [ ]:
# Count the number of urls in the message
twitter_df["url_count"] = twitter_df["message"].apply(lambda x: pre.get_url_count(x))

In [ ]:
# Remove urls from the message
twitter_df.loc[:, "message"] = twitter_df["message"].apply(lambda x: pre.remove_urls(x))

In [ ]:
twitter_df.head()

### Remove RT = Retweet, count and remove reply targets and mentions

In [ ]:
# Remove RT from the messages
twitter_df.loc[:, "message"] = twitter_df["message"].apply(lambda x: pre.remove_retweet(x))

In [ ]:
# Count mentions in the message
twitter_df["mention_count"] = twitter_df["message"].apply(lambda x: pre.get_mention_count(x))

In [ ]:
# Remove all mentions from the message
twitter_df.loc[:, "message"] = twitter_df["message"].apply(lambda x: pre.remove_mentions(x))

In [ ]:
# See if tweet is reply or not
twitter_df["is_reply"] = twitter_df["message"].apply(lambda x: pre.is_reply(x))

In [ ]:
# Remove reply target from message
twitter_df.loc[:, "message"] = twitter_df["message"].apply(lambda x: pre.remove_reply_target(x))

### Remove accents

In [ ]:
# Replace accented characters with their normal form
twitter_df.loc[:, "message"] = twitter_df["message"].apply(lambda x: pre.remove_accents(x))

### Remove special characters from text and extra spaces

In [ ]:
# Remove all characters that are not alphanumeric, hyphen or space from message
twitter_df.loc[:, "message"] = twitter_df["message"].apply(lambda x: pre.remove_special_characters(x))

In [ ]:
# Remove extra spaces from the message
twitter_df.loc[:, "message"] = twitter_df["message"].apply(lambda x: pre.remove_extra_spaces(x))